In [1]:
import time
import json
import urllib
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

import numpy as np
import pandas as pd

import re

In [2]:
url = "https://www.worldometers.info/coronavirus/#main_table"
resp = requests.get(url)
soup = BeautifulSoup(resp.text)

In [3]:
data = soup.find('div',attrs={'class':'main_table_countries_div'}).find("table").findAll('tr')
datalist = []
for td in data:
          x = td.text.replace("\n","~")
          datalist.append(x)
columns = "".join(datalist[0])
columns = list(columns.split("~"))

dic = {}
for n in columns :
  dic[n] =[""]

dataset = pd.DataFrame(dic)
dataset.rename(columns = {'Tests/': 'Tests/1M pop', 'Serious,Critical': 'SeriousCritical', 'Country,Other':'Country'}, inplace = True)
dataset = dataset.drop(columns = ["", "Continent" ,"1 Caseevery X ppl1 Deathevery X ppl1 Testevery X ppl",	"New Cases/1M pop"	,"New Deaths/1M pop",	"Active Cases/1M pop","1M pop"])
dataset.drop(0, inplace=True)

In [4]:
datalist = []
for td in range(9,len(data)):
          x = data[td].text.replace("\n","~")
          datalist.append(x)
          rs = "".join(datalist[td-9])
          datalist[td-9] = list(rs.split("~"))
          
          
for rv in range(1,9) :      
  del datalist[-1]

In [5]:
for x in range(0,len(datalist)):
  addraw = datalist[x]
  newaddraw = addraw[1:16] 

  toadd = {'#': newaddraw[0],
         'Country': newaddraw[1],
         'TotalCases': newaddraw[2],
         'NewCases':newaddraw[3],
         'TotalDeaths': newaddraw[4],
         'NewDeaths': newaddraw[5],
         'TotalRecovered': newaddraw[6],
         'NewRecovered': newaddraw[7],
         'ActiveCases': newaddraw[8],
         'SeriousCritical': newaddraw[9],
         'Tot Cases/1M pop': newaddraw[10],
         'Deaths/1M pop': newaddraw[11],
         'TotalTests': newaddraw[12],
         'Tests/1M pop': newaddraw[13],
         'Population': newaddraw[14]}
  
  df = pd.DataFrame.from_dict([toadd])
  dataset = pd.concat([dataset, df], ignore_index=True)

In [6]:
dataset.drop('#', axis = 1, inplace=True)
dataset.dropna(axis = 1, inplace=True)
dataset["TotalDeaths"] = dataset["TotalDeaths"].str.replace(" ", "")

In [7]:
dataset = dataset.replace("", 0)
dataset = dataset.replace("N/A", 0)
dataset = dataset.replace(" ", 0)

In [8]:
def comma_seperater(x):
    a = re.sub(' ', "", str(x))
    a = re.sub(",", '', a)
    b = int(a)
    return b

In [9]:
for i in dataset.columns[1:]:
    if "New" in i:
        pass
    else:
        dataset[i] = dataset[i].apply(comma_seperater)

In [10]:
dataset.sort_values("TotalCases", ascending=False, inplace= True)
dataset.reset_index(drop=True, inplace=True)
dataset.to_excel("corona.xlsx", index = False)